# LEITURA DOS CSV E ESCRITA EM ARQUIVO COMBINADO

In [ ]:
import pandas as pd

# Definir o diretório onde estão os arquivos .csv
# Use '.' para o diretório atual ou substitua pelo caminho da pasta
loja = ''
grupo = ''
empresa = ''
directory = 'D:/Dropbox/Documentos/Consultorias/'+empresa+'/'+loja+'/'+grupo+'/Presentes/Dados Brutos' 
currenty_time = pd.to_datetime('now').strftime('%Y-%m-%d %H:%M:%S').replace(":","_")
file_out = f"/Arquivos Combinados/combined_file_{currenty_time}.csv"
reports = 'D:/Dropbox/Documentos/Consultorias/'+empresa+'/'+loja+'/'+grupo+'/Presentes/Reports' 
# Exemplo de uso
combined_df, file_out  = uf.merge_csv_files_in_dataframe(directory, directory + file_out)
data_out = f'{directory}/Dados Saída/{currenty_time}.csv'

print(f'Todos os arquivos CSV foram combinados com sucesso e salvos em {file_out}!')

In [ ]:
import sys
sys.path.append('../analysis_pet_shop/') #  caminho relativo (supondo que a pasta modulos está um nível acima do notebook)
import importlib
import utils_graphics
import utils_import as ui
import utils_files as uf
importlib.reload(utils_graphics)

"""
import sys
sys.path.append('/caminho/para/a/pasta/modulos') # Caso o módulo esteja em uma pasta chamada modulos dentro de um diretório diferente:


# Usando o módulo importlib
import importlib.util
import sys

# Defina o caminho do módulo
module_path = '/caminho/para/a/pasta/modulos/meu_modulo.py'

# Carregue o módulo
spec = importlib.util.spec_from_file_location("meu_modulo", module_path)
meu_modulo = importlib.util.module_from_spec(spec)
sys.modules["meu_modulo"] = meu_modulo
spec.loader.exec_module(meu_modulo)

# Agora você pode usar o módulo
meu_modulo.sua_funcao()


# Usando o ! para executar comandos do sistema
!python3 /caminho/para/a/pasta/modulos/meu_modulo.py

"""


In [ ]:
ui.charge_reload()

In [ ]:
# apresentação do df_combined
df = combined_df
df.head(10)

In [ ]:
df.shape

# ANÁLISE DE VALORES REPETIDOS NO DATAFRAME

In [ ]:
import utils_prepared_data as upd
# Coluna a ser analisada
coluna = 'Numero da venda'

num_repeated = upd.count_occurences(df, coluna)

print("Valores repetidos e suas quantidades:")
print(num_repeated)

In [ ]:
df.shape

In [ ]:
df = upd.drop_repeated_column(df, coluna)

In [ ]:
df.head(10)

In [ ]:
df.shape

# AGRUPAMENTO DOS DADOS POR QUANTIDADE DE ATENDIMENTOS

In [ ]:
'''
groupby('coluna_de_agrupamento').size(): 
Agrupa o DataFrame pela coluna especificada e conta o número de ocorrências em cada grupo.

reset_index(name='contagem'): 
Restaura o índice, transformando-o em uma coluna e nomeia essa coluna de contagem como "contagem".
'''
column_date = 'Data Emissão'

# Agrupar pelo valor de 'coluna_de_agrupamento' e contar ocorrências
df_grouped = df.groupby(column_date).size().reset_index(name='Quantidade')

df_grouped.head(10)

In [ ]:
#column_date = 'Data Emissão'
# Agrupando o DataFrame pela coluna 'coluna_de_agrupamento' e calculando a média
#df_grouped = df.groupby(column_date).count().reset_index()

# Selecionando apenas a coluna desejada e atualizando o DataFrame
#df_grouped = df_grouped.loc[:, [column_date]]

#print(df_grouped)

In [ ]:
df_grouped.dtypes

In [ ]:
# Converte a coluna 'Data Emissão' de 'object' para 'datetime'
df_grouped['Data Emissão'] = pd.to_datetime(df_grouped['Data Emissão'], dayfirst=True)

In [ ]:
# Renomear as colunas
df_grouped.rename(columns={'Data Emissão': 'date_service', 'Quantidade': 'count'}, inplace=True)

In [ ]:
df_grouped.dtypes

In [ ]:
df_grouped.head(10)

In [ ]:
# Ordenar o DataFrame pela coluna 'data'
df_grouped_sorted = df_grouped.sort_values(by='date_service')

# Redefinir os índices
df_grouped_sorted.reset_index(drop=True, inplace=True)

In [ ]:
df_grouped_sorted.head(10)

# INSERÇÃO DE COLUNAS DERIVADAS DA DATA DO SERVIÇO

In [ ]:
import utils_date as ud

df = ud.get_all_columns(df_grouped_sorted,'date_service')
# Converte a coluna de datas para dias corridos
df['days_elapsed'] = (df['date_service'] - df['date_service'].min()).dt.days
# A coluna para mover para o início
coluna = df.pop('days_elapsed')
df.insert(0, 'days_elapsed', coluna)

In [ ]:
df.head(10)

# ARQUIVO COM DADOS DE SAÍDA

In [ ]:
# Salvar como arquivo CSV no computador
df.to_csv(data_out, index=False)  # index=False omite a coluna de índice

# ANÁLISE E VISUALIZAÇÃO DOS RESULTADOS

In [ ]:
# Identificação da análise

print(f"ANÁLISE DA QUANTIDADE DE ATENDIMENTOS {empresa.upper()} - {loja.upper()} - {grupo.upper()}")

# Estatísticas básicas gerais
import utils_statistics as us
import utils_graphics as ug
from docx import Document

# Crie um novo documento Word
doc = Document()

# Adicionando um título
doc.add_heading('Conteúdo do Notebook', 0)

# Suponha que você tenha um conteúdo que quer adicionar
output = """
Este é o conteúdo da célula que você deseja exportar para o Word.
Abaixo está um gráfico gerado durante a análise:
"""

# Adiciona o conteúdo textual ao documento
doc.add_paragraph(output)



image_path =  f'{reports}/{empresa}_{grupo}_{loja}_{currenty_time}.png'

df_statistics = us.get_descriptive_statistics(df,'count')
print(df_statistics.head())
df_statistics_day_of_week = us.get_grouped_descriptive_statistics(df,'day_of_week','count')
df_statistics_day_of_week = df_statistics_day_of_week[df_statistics_day_of_week['day_of_week'] != 'Sunday']
df_statistics_day_of_week.sort_values(by='mean', ascending = True)
print(df_statistics_day_of_week)

# Adiciona o conteúdo textual ao documento
doc.add_paragraph(str(df_statistics.head()))
doc.add_paragraph(str(df_statistics_day_of_week.head()))

ug.plot_weekday_means(df_statistics_day_of_week['day_of_week'], df_statistics_day_of_week['mean'],image_path)
# Adiciona a imagem ao documento
doc.add_picture(image_path)
# Salva o documento
doc.save(reports+'/output_notebook_com_imagem.docx')

ug.plot_boxplot(df,'day_of_week','count')
ug.plot_scatter_with_trendline(df,'date_service', 'count', 'day_of_week', 'Saturday')
ug.plot_scatter_with_trendline(df,'date_service', 'count', 'day_of_week', 'Friday')
ug.plot_scatter_with_trendline(df,'date_service', 'count', 'day_of_week', 'Monday' )
ug.plot_scatter_with_trendline(df,'date_service', 'count', 'day_of_week', 'Tuesday')
ug.plot_scatter_with_trendline(df,'date_service', 'count', 'day_of_week', 'Thursday')
ug.plot_scatter_with_trendline(df,'date_service', 'count', 'day_of_week', 'Wednesday')

# Calculando a média móvel agrupada por 'day_of_week'
df['moving_avg_day_of_week'] = df.groupby('day_of_week')['count'].transform(lambda x: us.moving_average(x, 7))

# Filtrando o DataFrame para obter a média móvel por dia da semana
df_monday = df[df['day_of_week'] == 'Monday'] [ ['days_elapsed', 'moving_avg_day_of_week'] ]
ug.plot_moving_average_by_day_of_week(df_monday,'days_elapsed', 'moving_avg_day_of_week', 'Monday')
df_tuesday = df[df['day_of_week'] == 'Tuesday'] [ ['days_elapsed', 'moving_avg_day_of_week'] ]
ug.plot_moving_average_by_day_of_week(df_tuesday,'days_elapsed', 'moving_avg_day_of_week', 'Tuesday')
df_wednesday = df[df['day_of_week'] == 'Wednesday'] [ ['days_elapsed', 'moving_avg_day_of_week'] ]
ug.plot_moving_average_by_day_of_week(df_wednesday,'days_elapsed', 'moving_avg_day_of_week', 'Wednesday')
df_thursday = df[df['day_of_week'] == 'Thursday'] [ ['days_elapsed', 'moving_avg_day_of_week'] ]
ug.plot_moving_average_by_day_of_week(df_thursday,'days_elapsed', 'moving_avg_day_of_week', 'Thursday')
df_friday = df[df['day_of_week'] == 'Friday'] [ ['days_elapsed', 'moving_avg_day_of_week'] ]
ug.plot_moving_average_by_day_of_week(df_friday,'days_elapsed', 'moving_avg_day_of_week', 'Friday')
df_saturday = df[df['day_of_week'] == 'Saturday'] [ ['days_elapsed', 'moving_avg_day_of_week'] ]
ug.plot_moving_average_by_day_of_week(df_saturday,'days_elapsed', 'moving_avg_day_of_week', 'Saturday')
ug.plot_pareto(df, 'day_of_week', 'count' )
df['moving_avg'] = us.moving_average(df['count'],7)

# Média móvel segmentada por períodos
ug.plot_moving_average_with_trendline(df['count'],df['moving_avg'],7,30,365*3,True)
ug.plot_moving_average_with_trendline(df['count'],df['moving_avg'],7,30, 365*2, True)
ug.plot_moving_average_with_trendline(df['count'],df['moving_avg'],7,30, 365+240, True)
ug.plot_moving_average_with_trendline(df['count'],df['moving_avg'],7,30, 365+180, True)
ug.plot_moving_average_with_trendline(df['count'],df['moving_avg'],7,30, 365, True)
ug.plot_moving_average_with_trendline(df['count'],df['moving_avg'],7,30, 365+90, True)
ug.plot_moving_average_with_trendline(df['count'],df['moving_avg'],7,30, 365, True)
ug.plot_moving_average_with_trendline(df['count'],df['moving_avg'],7,30, 270, True)
ug.plot_moving_average_with_trendline(df['count'],df['moving_avg'],7,30, 180, True)
ug.plot_moving_average_with_trendline(df['count'],df['moving_avg'],7,30, 90, True)
ug.plot_moving_average_with_trendline(df['count'],df['moving_avg'],7,30, 30, True)

# Estatísticas por mês do ano
df_statistics_day_of_week = us.get_grouped_descriptive_statistics(df,'year_month','count')
df_statistics_day_of_week.sort_values(by='year_month', ascending = True)

# Comparativo da quantidade de atendimentos por mês do ano
ug.plot_column_chart_with_legend_stacked(df,'month','count','year')
ug.plot_column_chart_with_legend_side_by_side(df,'month','count','year')